In [7]:
import numpy as np
from hashlib import md5

def hash_trick(tokens, n_dimensions=1000):
    """
    Implements the hashing trick for text feature representation.
    
    Args:
        tokens: List of strings (words or n-grams)
        n_dimensions: Size of the feature vector
    
    Returns:
        A feature vector of size n_dimensions
    """
    # Initialize the feature vector with zeros
    feature_vector = np.zeros(n_dimensions)
    
    # For each token in the input
    for token in tokens:
        # Hash the token to get an integer
        hash_value = int(md5(token.encode('utf-8')).hexdigest(), 16)
        
        # Map the hash value to a position in the feature vector
        position = hash_value % n_dimensions
        
        # Increment the count at that position
        feature_vector[position] += 1
    
    return feature_vector

# Example usage
text = "this is a simple example of the hashing trick in action"
tokens = text.lower().split()

# Create feature vector with 20 dimensions
feature_vector = hash_trick(tokens, n_dimensions=20)

print("Tokens:", tokens)
print("Feature vector:", feature_vector)

# Let's see what happens with two different texts that share some words
text2 = "another example of the hashing trick"
tokens2 = text2.lower().split()
feature_vector2 = hash_trick(tokens2, n_dimensions=20)

print("\nTokens:", tokens2)
print("Feature vector:", feature_vector2)

# Some common words will map to the same position in both vectors
print("\nPositions where collisions might occur:")
for i in range(len(feature_vector)):
    if feature_vector[i] > 0 and feature_vector2[i] > 0:
        print(f"Position {i}: {feature_vector[i]} vs {feature_vector2[i]}")

Tokens: ['this', 'is', 'a', 'simple', 'example', 'of', 'the', 'hashing', 'trick', 'in', 'action']
Feature vector: [2. 1. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 1. 1. 0. 1. 0. 2.]

Tokens: ['another', 'example', 'of', 'the', 'hashing', 'trick']
Feature vector: [2. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]

Positions where collisions might occur:
Position 0: 2.0 vs 2.0
Position 1: 1.0 vs 1.0
Position 15: 1.0 vs 1.0
Position 19: 2.0 vs 1.0


In [6]:
hash_trick(['hello', 'world'], n_dimensions=20)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0.])

In [ ]:
import torch
import torch.nn as nn

class HashEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_hashes, pool_size):
        super(HashEmbedding, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.num_hashes = num_hashes
        self.pool_size = pool_size

        # Shared pool of embedding vectors
        self.embedding_pool = nn.Embedding(pool_size, embedding_dim)

        # Importance parameters for each word
        self.importance_weights = nn.Embedding(vocab_size, num_hashes)

        # Hash functions (random but fixed). 
        # technically, this is really big with pool_size * vocab_size but it's just for the sake of easy implementation.
        self.hash_functions = [
            torch.randint(0, pool_size, (vocab_size,)) for _ in range(num_hashes)]

    def forward(self, word_ids):
        # Get importance weights for the input words
        weights = self.importance_weights(word_ids)  # Shape: (batch_size, num_hashes)

        # Get component vectors using hash functions
        component_vectors = []
        for i in range(self.num_hashes):
            # Apply hash function to get indices into the shared pool
            indices = self.hash_functions[i][word_ids]  # Shape: (batch_size,)
            # Lookup vectors from the shared pool
            vectors = self.embedding_pool(indices)  # Shape: (batch_size, embedding_dim)
            component_vectors.append(vectors)

        # Stack component vectors and compute weighted sum
        component_vectors = torch.stack(component_vectors, dim=1)  # Shape: (batch_size, num_hashes, embedding_dim)
        weights = weights.unsqueeze(-1)  # Shape: (batch_size, num_hashes, 1)
        final_embeddings = (weights * component_vectors).sum(dim=1)  # Shape: (batch_size, embedding_dim)

        return final_embeddings

# Example usage
vocab_size = 10000  # Number of unique words
embedding_dim = 50   # Dimension of each embedding vector
num_hashes = 2       # Number of hash functions
pool_size = 1000     # Size of the shared embedding pool

# Create hash embedding layer
hash_embedding = HashEmbedding(vocab_size, embedding_dim, num_hashes, pool_size)

# Input: batch of word IDs (e.g., [3, 7, 2])
word_ids = torch.tensor([3, 7, 2])

# Get embeddings
embeddings = hash_embedding(word_ids)
print(embeddings.shape)  # Output: (3, 50)

torch.Size([3, 50])


In [15]:
hash_embedding.hash_functions[0].shape

torch.Size([10000])

In [13]:
embeddings = hash_embedding(word_ids)
embeddings

tensor([[ 2.0106e-01, -9.4563e-01,  1.1096e+00,  1.2103e+00,  4.2908e-01,
         -1.4054e-01, -2.9341e-01, -1.4251e+00, -1.0027e+00,  6.0199e-01,
          7.8202e-01,  8.9964e-01, -6.9072e-01,  8.4406e-01,  3.8960e-02,
          5.5408e-01, -2.1158e-01, -1.0466e+00, -7.9575e-01, -7.2971e-01,
         -2.2697e-01, -5.1622e-01, -3.2555e-01, -1.3523e-01, -4.0508e-01,
          7.6875e-01,  8.1700e-01,  8.0329e-01,  8.3822e-01, -9.8230e-01,
         -7.5856e-02,  7.7942e-01, -4.5099e-01,  6.5127e-01, -5.1193e-02,
          2.0867e-01, -5.6362e-01,  2.1198e-01,  1.9371e-01, -1.4677e+00,
         -3.5016e-01,  2.7238e-01,  9.2124e-01,  3.3926e-01, -8.9065e-01,
          9.0552e-02,  5.5217e-01,  1.9165e+00,  4.4012e-01, -7.9510e-03],
        [-9.9130e-01,  2.1554e+00,  9.0176e-01,  2.4565e+00,  2.6315e+00,
         -3.7404e+00,  3.5755e-01,  1.0505e-01, -5.2969e-01,  6.0918e-01,
         -6.5580e-01, -1.4672e+00,  5.2210e-01,  6.6820e-01, -2.7233e+00,
          3.2347e+00,  2.5359e+00, -2